In [2]:
import pandas as pd
import numpy as np

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

from keras_preprocessing.text import Tokenizer

import matplotlib.pyplot as plt

In [3]:
import lstm_preprocess

In [4]:
import os
import tensorflow as tf
import random
# fix the random seed for tensorflow models
os.environ['TF_DETERMINISTIC_OPS'] = '1' 
SEED = 39
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# set to use flexible GPU resources  
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [5]:
print(f"Num GPUs Available: {len(tf.config.list_physical_devices('GPU'))}.")

Num GPUs Available: 1.


## Read the data

In [6]:
data = pd.read_csv("./icd_demos_vitals.csv")

data.drop("Unnamed: 0", axis=1, inplace=True)

In [7]:
#either keep the patients with 24 hours of admission, or change 23 to sth smaller to include patients with less time steps
df = lstm_preprocess.pad(data, 23, 24, 0)

df

,mortality,heartrate,sysbp,diasbp,meanbp,resprate,tempc,spo2,glucose,hadm_id,...,18,19,20,F,M,18-25,25-45,45-65,65-89,89+
0,1.0,0.271698,0.331190,0.248120,0.251678,0.144928,0.000000,0.93,0.0,100061.0,...,0.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.283019,0.299035,0.240602,0.241611,0.130435,0.000000,0.91,0.0,100061.0,...,0.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.275472,0.347267,0.229323,0.238255,0.144928,0.000000,0.94,0.0,100061.0,...,0.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.283019,0.344051,0.221805,0.238255,0.260870,0.000000,0.95,0.0,100061.0,...,0.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.298113,0.379421,0.323308,0.305369,0.231884,0.000000,0.95,0.0,100061.0,...,0.0,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53131,0.0,0.305660,0.331190,0.203008,0.224832,0.246377,0.872200,0.94,0.0,199984.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
53132,0.0,0.298113,0.356913,0.195489,0.218121,0.275362,0.000000,0.00,0.0,199984.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
53133,0.0,0.279245,0.382637,0.233083,0.251678,0.333333,0.000000,0.00,0.0,199984.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
53134,0.0,0.305660,0.389068,0.236842,0.261745,0.231884,0.000000,0.00,0.0,199984.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
#remove unnecessary columns
COLUMNS = lstm_preprocess.delete_columns(df)

COLUMNS

['heartrate',
 'sysbp',
 'diasbp',
 'meanbp',
 'resprate',
 'tempc',
 'spo2',
 'glucose',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 'F',
 'M',
 '18-25',
 '25-45',
 '45-65',
 '65-89',
 '89+']

In [23]:
hadm_idx = df['hadm_id'].values.reshape(-1, 24, 1)[:, 0, 0]
hadm_idx

array([100061., 100087., 100104., ..., 199976., 199981., 199984.])

In [9]:
#reshape the matrix to the appropriate format
MATRIX = df[COLUMNS+['mortality']].values
MATRIX = MATRIX.reshape(int(MATRIX.shape[0]/24),24,MATRIX.shape[1])

In [10]:
bool_matrix = (~MATRIX.any(axis=2))
MATRIX[bool_matrix] = np.nan
MATRIX = lstm_preprocess.ZScoreNormalize(MATRIX)

## restore 3D shape to boolmatrix for consistency
bool_matrix = np.isnan(MATRIX)
MATRIX[bool_matrix] = 0 
   
#permutation = np.random.permutation(MATRIX.shape[0])
#MATRIX = MATRIX[permutation]
#bool_matrix = bool_matrix[permutation]

# X_MATRIX = MATRIX[:,:,0:-1]
X_MATRIX = MATRIX[:,:,0:8] # only use the first 8 temporal features, ignoring demographic data for now
Y_MATRIX = MATRIX[:,:,-1]
sc = MinMaxScaler()
#x_bool_matrix = bool_matrix[:,:,0:-1]
#y_bool_matrix = bool_matrix[:,:,-1]

(2214, 24)
(2214, 24, 35)
(35,)
(35,)
(2214, 24, 35)
(2214, 24, 35)
(2214, 24, 1)


/data0/home/dsv/zhwa9764/Projects/KTH_PROJECT_MIMIC_2021/data_extraction/lstm_preprocess.py:78: RuntimeWarning: invalid value encountered in true_divide
  x_matrix = x_matrix / stds


In [11]:
pd.value_counts(Y_MATRIX[:, 0])

0.0    1819
1.0     395
dtype: int64

In [12]:
#train, validation, test split
tt_split = 0.7 
val_percentage = 0.8

train_tail_idx = int(tt_split*X_MATRIX.shape[0])
val_tail_idx = int(val_percentage*X_MATRIX.shape[0])

In [24]:
# extract the admission idx for train/validation/test, used to extract text patient data for prediction 
train_hadm_idx = hadm_idx[:train_tail_idx]
val_hadm_idx = hadm_idx[train_tail_idx:val_tail_idx]
test_hadm_idx = hadm_idx[val_tail_idx::]

In [26]:
X_TRAIN = X_MATRIX[0:train_tail_idx, :, :]
Y_TRAIN = Y_MATRIX[0:train_tail_idx, 0]
# Y_TRAIN = Y_TRAIN.reshape(Y_TRAIN.shape[0], Y_TRAIN.shape[1], 1)

X_VAL = X_MATRIX[train_tail_idx:val_tail_idx]
Y_VAL = Y_MATRIX[train_tail_idx:val_tail_idx, 0]
# Y_VAL = Y_VAL.reshape(Y_VAL.shape[0], Y_VAL.shape[1], 1)
"""
x_val_boolmat = x_bool_matrix[int(tt_split*x_bool_matrix.shape[0]):int(val_percentage*x_bool_matrix.shape[0])]
y_val_boolmat = y_bool_matrix[int(tt_split*y_bool_matrix.shape[0]):int(val_percentage*y_bool_matrix.shape[0])]
y_val_boolmat = y_val_boolmat.reshape(y_val_boolmat.shape[0],y_val_boolmat.shape[1],1)
"""
X_TEST = X_MATRIX[val_tail_idx::]
Y_TEST = Y_MATRIX[val_tail_idx::, 0]
# Y_TEST = Y_TEST.reshape(Y_TEST.shape[0], Y_TEST.shape[1], 1)
"""
x_test_boolmat = x_bool_matrix[int(val_percentage*x_bool_matrix.shape[0])::]
y_test_boolmat = y_bool_matrix[int(val_percentage*y_bool_matrix.shape[0])::]
y_test_boolmat = y_test_boolmat.reshape(y_test_boolmat.shape[0],y_test_boolmat.shape[1],1)

X_TEST[x_test_boolmat] = 0
Y_TEST[y_test_boolmat] = 0
"""

'\nx_test_boolmat = x_bool_matrix[int(val_percentage*x_bool_matrix.shape[0])::]\ny_test_boolmat = y_bool_matrix[int(val_percentage*y_bool_matrix.shape[0])::]\ny_test_boolmat = y_test_boolmat.reshape(y_test_boolmat.shape[0],y_test_boolmat.shape[1],1)\n\nX_TEST[x_test_boolmat] = 0\nY_TEST[y_test_boolmat] = 0\n'

In [40]:
no_feature_cols = X_TRAIN.shape[2]
no_feature_cols

8

## Load text data

In [27]:
text_data = pd.read_csv('./texts.csv')

text_data.head()

,Unnamed: 0,text,subject_id,hadm_id,mortality
0,0,sinus bradycardia av conduction delay qt inter...,67393,146953.0,0
1,1,baseline artifact sinus rhythm right bundle br...,9142,198248.0,1
2,2,atrial fibrillation possible flutter ventricul...,73608,162231.0,0
3,3,mobitz ii second degree av block nonconducted ...,29016,153345.0,0
4,4,sinus rhythm normal tracing previous tracing a...,12834,107726.0,0


In [ ]:
# X, y = text_data['text'], text_data['mortality']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=39, stratify=y)

In [28]:
train_data = text_data[text_data['hadm_id'].isin(set(train_hadm_idx))]
val_data = text_data[text_data['hadm_id'].isin(set(val_hadm_idx))]
test_data = text_data[text_data['hadm_id'].isin(set(test_hadm_idx))]

X_train, y_train = train_data['text'],  train_data['mortality']
X_val, y_val = val_data['text'],  val_data['mortality']
X_test, y_test = test_data['text'],  test_data['mortality']

In [29]:
# find the joint hadm ids for train/test/val
join_train_hadm_idx = np.unique(train_data['hadm_id'])
join_val_hadm_idx = np.unique(val_data['hadm_id'])
join_test_hadm_idx = np.unique(test_data['hadm_id'])

In [30]:
# modify the time series data to have same hadm id as text data
X_TRAIN = X_TRAIN[np.in1d(train_hadm_idx, join_train_hadm_idx)]
Y_TRAIN = Y_TRAIN[np.in1d(train_hadm_idx, join_train_hadm_idx)]

X_VAL = X_VAL[np.in1d(val_hadm_idx, join_val_hadm_idx)]
Y_VAL = Y_VAL[np.in1d(val_hadm_idx, join_val_hadm_idx)]

X_TEST = X_TEST[np.in1d(test_hadm_idx, join_test_hadm_idx)]
Y_TEST = Y_TEST[np.in1d(test_hadm_idx, join_test_hadm_idx)]


In [31]:
X_train.shape

(1529,)

In [32]:
X_TRAIN.shape

(1529, 24, 8)

In [33]:
NUM_WORDS = 3000

# Tokenize the train text
train_text = X_train.to_numpy()

tokenizer = Tokenizer(
    num_words=NUM_WORDS, 
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
    lower=True,
    split=" ",
    char_level=False,
    oov_token='<unk>',
    document_count=0
)

tokenizer.fit_on_texts(train_text)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

In [34]:
MAX_LEN = 1000

train_seqs = tokenizer.texts_to_sequences(train_text)
train_seqs = keras.preprocessing.sequence.pad_sequences(train_seqs, maxlen=MAX_LEN, padding='post')

train_labels = y_train.to_numpy().flatten()

valid_text = X_val.to_numpy()
valid_seqs = tokenizer.texts_to_sequences(valid_text)
valid_seqs = keras.preprocessing.sequence.pad_sequences(valid_seqs, maxlen=MAX_LEN, padding='post')

valid_labels = y_val.to_numpy().flatten()

In [35]:
train_seqs

array([[ 147,  807,  976, ...,  149,    1,    2],
       [  14, 1403,   82, ...,  108,  764,  767],
       [   7,  151, 1162, ...,   63,  712,  332],
       ...,
       [ 113,   74, 1416, ...,    0,    0,    0],
       [ 113,   74,  831, ...,    0,    0,    0],
       [ 274,  508, 1523, ...,    0,    0,    0]], dtype=int32)

In [36]:
train_seqs.shape

# (30455, 606752) without preprocessing/limiting and truncating

(1529, 1000)

## Use the composite model

In [37]:
def CompositeModel(n_timesteps, n_features, text_input_size): # n_features2=1 for text data
    # classifier 1 (for time series):
    inputs1 = keras.Input(shape=(n_timesteps, n_features))
    output1 = keras.layers.LSTM(64, activation='tanh')(inputs1)

    classifier1 = keras.Model(inputs1, output1, name="classifier1")

    # classifier 2 (for text data)
    inputs2 = keras.Input(text_input_size,)
    x = keras.layers.Embedding(input_dim=NUM_WORDS, output_dim=64, input_length=text_input_size)(inputs2)
    output2 = keras.layers.LSTM(64, activation='tanh')(x)

    classifier2 = keras.models.Model(inputs2, output2, name="classifier2")

    # final prediction
    combined = keras.layers.concatenate([classifier1.output, classifier2.output])
    # combined outputs
    x = keras.layers.Dense(2, activation="relu")(combined)
    outputs3 = keras.layers.Dense(1, activation="sigmoid")(x)

    composite_model = keras.models.Model([classifier1.input, classifier2.input], outputs3)
    
    return composite_model

In [41]:
composite_model = CompositeModel(n_timesteps=24, n_features=no_feature_cols, text_input_size=MAX_LEN)

optimizer = keras.optimizers.Adam(lr=0.0001)
composite_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

composite_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 24, 8)]      0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1000, 64)     192000      input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 64)           18688       input_1[0][0]                    
______________________________________________________________________________________________

In [48]:
#TODO: plot the model structure
keras.utils.plot_model(composite_model, show_shapes=True)


('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [49]:
X_TRAIN.shape

(1529, 24, 8)

In [50]:
train_seqs.shape

(1529, 1000)

In [52]:
# Define the early stopping criteria
early_stopping_accuracy = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True) # patient = 5 or 10 doesn't guarantee find an optimal

# Train the model
# reset_seeds()
classifier_history2 = composite_model.fit([X_TRAIN, train_seqs], 
          y_train, 
          epochs=50,
          batch_size=128,
          shuffle=True, 
          verbose=True, 
          validation_data=([X_VAL, valid_seqs], y_val),
          callbacks=[early_stopping_accuracy])

Epoch 1/50
12/12 [==============================] - 4s 103ms/step - loss: 0.6757 - accuracy: 0.7637 - val_loss: 0.6656 - val_accuracy: 0.7717
Epoch 2/50
12/12 [==============================] - 1s 62ms/step - loss: 0.6637 - accuracy: 0.8098 - val_loss: 0.6528 - val_accuracy: 0.8037
Epoch 3/50
12/12 [==============================] - 1s 63ms/step - loss: 0.6500 - accuracy: 0.8117 - val_loss: 0.6377 - val_accuracy: 0.8082
Epoch 4/50
12/12 [==============================] - 1s 62ms/step - loss: 0.6361 - accuracy: 0.8221 - val_loss: 0.6182 - val_accuracy: 0.8037
Epoch 5/50
12/12 [==============================] - 1s 64ms/step - loss: 0.6169 - accuracy: 0.8263 - val_loss: 0.5914 - val_accuracy: 0.8082
Epoch 6/50
12/12 [==============================] - 1s 63ms/step - loss: 0.5905 - accuracy: 0.8148 - val_loss: 0.5428 - val_accuracy: 0.8082
Epoch 7/50
12/12 [==============================] - 1s 62ms/step - loss: 0.5265 - accuracy: 0.8278 - val_loss: 0.4690 - val_accuracy: 0.8082
Epoch 8/50
1

In [55]:
y_pred = composite_model.predict([X_VAL, valid_seqs])
y_pred_classes = np.array([1 if pred > 0.5 else 0 for pred in y_pred])

# classification report
acc = accuracy_score(y_true=Y_VAL, y_pred=y_pred_classes)
print(acc)

confusion_matrix_df = pd.DataFrame(
        confusion_matrix(y_true=Y_VAL, y_pred=y_pred_classes, labels=[1, 0]),
        index=['True:pos', 'True:neg'], 
        columns=['Pred:pos', 'Pred:neg']
    )
print(confusion_matrix_df)

print(classification_report(y_true=Y_VAL, y_pred=y_pred_classes))

0.7990867579908676
          Pred:pos  Pred:neg
True:pos         0        42
True:neg         2       175
              precision    recall  f1-score   support

         0.0       0.81      0.99      0.89       177
         1.0       0.00      0.00      0.00        42

    accuracy                           0.80       219
   macro avg       0.40      0.49      0.44       219
weighted avg       0.65      0.80      0.72       219

